In [3]:
%reload_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
from stockMarket.core import get_tickers_from_index
from stockMarket.api import populate_contracts

tickers = get_tickers_from_index('sp500')
# contracts = populate_contracts(tickers)

In [27]:
from stockMarket.sec_edgar.downloader import Downloader

tickers = [ticker.replace(".", "-") for ticker in tickers]

dl = Downloader(tickers=tickers)
dl.download_10k_reports(fast_download=True)
dl.download_financial_statements_xlsx(from_year=2014)

  0%|          | 0/503 [00:00<?, ?it/s]/home/jag/Trading/stockMarket/stockMarket/sec_edgar/downloader.py:69: UserWarning: Failed to download https://www.sec.gov/Archives/edgar/data/0000900075/000104746997001831/Financial_Report.xlsx for CPRT
  if response.status_code != 200:
/home/jag/Trading/stockMarket/stockMarket/sec_edgar/downloader.py:69: UserWarning: Failed to download https://www.sec.gov/Archives/edgar/data/0000900075/000091205796023870/Financial_Report.xlsx for CPRT
  if response.status_code != 200:
  0%|          | 1/503 [00:03<31:07,  3.72s/it]/home/jag/Trading/stockMarket/stockMarket/sec_edgar/downloader.py:69: UserWarning: Failed to download https://www.sec.gov/Archives/edgar/data/0000101829/000010182999000003/Financial_Report.xlsx for RTX
  if response.status_code != 200:
/home/jag/Trading/stockMarket/stockMarket/sec_edgar/downloader.py:69: UserWarning: Failed to download https://www.sec.gov/Archives/edgar/data/0000101829/000010182995000012/Financial_Report.xlsx for RTX
  

In [23]:
from stockMarket.sec_edgar.financialReportReader import FinancialReportReader

reader = FinancialReportReader("../data/sec-edgar-financials/AAPL/10-K/data/sec-edgar-filings/AAPL/10-K/0000320193-17-000070/Financial_Report.xlsx")
reader.read()

FileNotFoundError: [Errno 2] No such file or directory: '../data/sec-edgar-financials/AAPL/10-K/data/sec-edgar-filings/AAPL/10-K/0000320193-17-000070/Financial_Report.xlsx'